In [1]:
###########################################################
###########################################################
###########################################################

## 自变量 挖矿节点 难度目标
## 因变量 出块时间

## 目标：找到一组合适的 PID 参数，使得每挖一个块，做一次难度调节
##      使得连续生成 1000 个块的方差较小
##      多组PID参数做比较 原始难度调整策略做比较

## 使用hash算法 sha1, 一共 160位，难度前导0最多159位

## 目标挖矿平均时间 60s， 记录每个新块的产生时间

## CPU: 2.3 GHz Intel Core i9, 8 - core, use muti-process to simulate the mining nodes

## the root hash of merkle tree is selected by random.

###########################################################
###########################################################
###########################################################

In [17]:
## Support Functions
import hashlib
def sha1(obj: str):
    return hashlib.sha1(str(obj).encode('utf-8')).hexdigest()

In [18]:
## Block
class Block:
    duration = 0
    
    def __init__(self, height, pre_hash, merkle, nouce, transactions):
        self.height = height
        self.pre_hash = pre_hash
        self.merkle = merkle
        self.nouce = nouce
        self.transactions = transactions

    def getHash(self):
        return sha1(self)
    
    def __repr__(self):
        return str(self.pre_hash) + str(self.height) + str(self.merkle) + str(self.nouce)
    
    def printJSON(self):
        # DFS or BFS
        print("Hash:", self.getHash(), "Height:", self.height, "Pre_Hash:", self.pre_hash, "Merkle Hash:", self.merkle, "nouce:", self.nouce)
        pass

In [22]:
## mine difficaulty
## 40 - 4 = 36
## hash value should smaller than DIFFICULTY
DIFFICULTY = 0x0000FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
print('Mine DIFFICULTY:', DIFFICULTY)
len("000000FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF")

Mine DIFFICULTY: 22300745198530623141535718272648361505980415


40

In [20]:
import numpy as np
import matplotlib.pyplot as plt
def visualization(blocks):
    x = range(len(blocks))
    y = [block.duration for block in blocks]
    x_standard = range(len(blocks))
    y_standard = [60 for i in x_standard]
    plt.figure(figsize=(8,4)) 
    l1=plt.plot(x,y,'r--',label='Mining')
    l2=plt.plot(x_standard,y_standard,'g--',label='Standard')
    plt.plot(x,y,'ro-',x_standard,y_standard,'g+-')
    plt.xlabel("Height")
    plt.ylabel("Time(s)")
    plt.legend()
    plt.title("Time Analysis")

# visualization([{"duration": 12},{"duration": 40}])

In [36]:
## PID ajustment
## U(t) = Kp*error(t) + Ki*integration(error(t)) + Kd*(error(t) - error(t-1))
## Initialize kp = 1.5  Ki = 1 Kd = 1, more epochs to find the best coefficient
## target = 60s
## error(t) = 60s - real_time
## new_Diffculty = old_Diffculty + U(t)
## old_Difficulty = 5 means 00000 + FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF

import math
kp = 0.5
ki = 0.1
kd = 0.1
# kp = 0.5, ki = 0.2, kd = 0.1
# kp = 0.5, ki = 0.3, kd = 0.1
# kp = 0.5, ki = 0.4, kd = 0.1

def adjustmentDifficulty(old_Difficulty,height):
    old_Difficulty = math.log2(old_Difficulty)
    print("old_Difficulty:",old_Difficulty)
    ut = kp*errors[height] + ki*errors_sum + kd*(errors[height] - errors[height-1])
    print("ut:", ut) 
    new_Difficulty = old_Difficulty - ut
    print("new_Difficulty:",new_Difficulty)
    return math.pow(2,  new_Difficulty)  

def adjustmentDifficulty2(old_Difficulty):
    ut = kp*errors[-1] + ki*errors_sum + kd*(errors[-1] - errors[-2])
    new_Difficulty = old_Difficulty - ut
    return new_Difficulty

# print(adjustmentDifficulty(DIFFICULTY))
    

In [37]:
## Start mining
import random
import sys
import os
from datetime import datetime
from multiprocessing import Process, Pool, Queue, Manager

epochs = 100
blocks = []
errors = [0]
errors_sum = 0
pre_hash = '0'
height = 1
max_nounce = sys.maxsize

# 100 blocks
nodes_numbers = [0,25,12,12,15,19,18,19,12,25,23,25,24,22,4,15,16,17,23,6,7,2,18,19,12,7,16,24,14,14,6,7]
#                  25,12,12,15,19,18,19,12,25,23,25,24,22,4,15,16,17,23,6,7,2,18,19,12,7,16,24,14,14,6,7,
#                  25,12,12,15,19,18,19,12,25,23,25,24,22,4,15,16,17,23,6,7,2,18,19,12,7,16,24,14,14,6,7,
#                  12,12,15,19,19,24,12,7,8,25]

def long_time_task(i, queue, start, height, pre_hash, merkle, DIFFICULTY):
#     print('Run task %s (%s)...' % (i, os.getpid()))  
    start_time = datetime.now()
    while queue.empty():
#         print("process: %s nounce: %s" % (i, start))
        block = Block(height, pre_hash, merkle, start, [])
        if int(block.getHash(),16) < DIFFICULTY:
            block.printJSON()
            queue.put(start)
            break
        else:
            start += 1
#     end_time = datetime.now()
#     duration = end_time - start_time
#     duration = duration.total_seconds()
#     print(duration)
    
while height < len(nodes_numbers):
    print('Epoch %s, Parent process %s, mining...' % (height, os.getpid()))
    merkle = sha1(random.random())
    nodes_number = nodes_numbers[height]
    unit_nounce = max_nounce // nodes_number
    p = Pool()
    queue = Manager().Queue()
    start_time = datetime.now()

    for i in range(nodes_number):
        start = i * unit_nounce # nounce
        p.apply_async(long_time_task, args = (i, queue, start, height, pre_hash, merkle, DIFFICULTY))  
    p.close()
    p.join()
    
    end_time = datetime.now()
    duration = end_time - start_time
    duration = duration.total_seconds()
    print('duration time: %s' % duration)
    
    block = Block(height, pre_hash, merkle, queue.get(True), [])
    block.duration = duration
    blocks.append(block)
    errors.append(60 - duration)
    errors_sum += duration
    
    print('errors', errors, "errors_sum", errors_sum)
    DIFFICULTY = adjustmentDifficulty(DIFFICULTY,height)

    height += 1
    pre_hash = block.getHash()

visualization(blocks)

Epoch 1, Parent process 42773, mining...


Process ForkPoolWorker-738:
Process ForkPoolWorker-737:
Process ForkPoolWorker-735:
Process ForkPoolWorker-734:
Process ForkPoolWorker-744:
Process ForkPoolWorker-736:
Process ForkPoolWorker-733:
Process ForkPoolWorker-745:


KeyboardInterrupt: 

Process ForkPoolWorker-739:
Process ForkPoolWorker-741:
Process ForkPoolWorker-743:
Process ForkPoolWorker-742:
Process ForkPoolWorker-746:
Process ForkPoolWorker-732:
Process ForkPoolWorker-731:
Process ForkPoolWorker-740:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/fourteen/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/fourteen/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/fourteen/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/fourteen/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/fourteen/anaconda3/lib/python3.7/multiprocessing/process.py", li

  File "<ipython-input-37-41f1bd566434>", line 25, in long_time_task
    while queue.empty():
  File "<string>", line 2, in empty
  File "<ipython-input-37-41f1bd566434>", line 25, in long_time_task
    while queue.empty():
  File "<string>", line 2, in empty
  File "<ipython-input-37-41f1bd566434>", line 25, in long_time_task
    while queue.empty():
  File "<ipython-input-37-41f1bd566434>", line 25, in long_time_task
    while queue.empty():
  File "<string>", line 2, in empty
  File "<ipython-input-37-41f1bd566434>", line 25, in long_time_task
    while queue.empty():
  File "/Users/fourteen/anaconda3/lib/python3.7/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "<ipython-input-37-41f1bd566434>", line 25, in long_time_task
    while queue.empty():
  File "<ipython-input-37-41f1bd566434>", line 25, in long_time_task
    while queue.empty():
  File "/Users/fourteen/anaconda3/lib/python3.7/multiprocessing/connection.py", line 250, in recv
    buf =

  File "/Users/fourteen/anaconda3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/Users/fourteen/anaconda3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
  File "/Users/fourteen/anaconda3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/Users/fourteen/anaconda3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/Users/fourteen/anaconda3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
  File "/Users/fourteen/anaconda3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt


In [149]:
from datetime import datetime
import time
start_time = datetime.now()
time.sleep(2)
end_time = datetime.now()
a = end_time - start_time
a.total_seconds()

2.005374

In [150]:
from multiprocessing import Process, Pool
import os

def run_proc(name):
    print('Run child process %s (%s)...' % (name, os.getpid()))
    
print('Parent process %s.' % os.getpid())
p = Process(target=run_proc, args=('test',))
print('Child process will start.')
p.start()
p.join()
print('Child process end.')

Parent process 6145.
Child process will start.
Child process end.


In [67]:
from multiprocessing import Pool
import os, time, random

def long_time_task(name):
    print('Run task %s (%s)...' % (name, os.getpid()))
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print('Task %s runs %0.2f seconds.' % (name, (end - start)))

print('Parent process %s.' % os.getpid())
p = Pool(8)
for i in range(8):
    p.apply_async(long_time_task, args=(i,))
print('Waiting for all subprocesses done...')
p.close()
p.join()
print('All subprocesses done.')

Parent process 6145.
Run task 2 (11449)...
Run task 0 (11447)...
Run task 3 (11450)...
Run task 1 (11448)...
Run task 4 (11451)...
Run task 6 (11453)...
Run task 5 (11452)...
Run task 7 (11454)...
Task 6 runs 0.07 seconds.
Waiting for all subprocesses done...
Task 2 runs 0.50 seconds.
Task 5 runs 0.57 seconds.
Task 3 runs 0.62 seconds.
Task 0 runs 0.87 seconds.
Task 1 runs 1.03 seconds.
Task 7 runs 2.76 seconds.
Task 4 runs 3.00 seconds.
All subprocesses done.


In [11]:
from multiprocessing import Process, Queue
import os, time, random

# 写数据进程执行的代码:
def write(q):
    print('Process to write: %s' % os.getpid())
    for value in ['A', 'B', 'C']:
        print('Put %s to queue...' % value)
        q.put(value)
        time.sleep(random.random())

# 读数据进程执行的代码:
def read(q):
    print('Process to read: %s' % os.getpid())
    while True:
        value = q.get(True)
        print('Get %s from queue.' % value)

# 父进程创建Queue，并传给各个子进程：
q = Queue()
pw = Process(target=write, args=(q,))
pr = Process(target=read, args=(q,))
# 启动子进程pw，写入:
pw.start()
# 启动子进程pr，读取:
pr.start()
# 等待pw结束:
pw.join()
# pr进程里是死循环，无法等待其结束，只能强行终止:
pr.terminate()

Process to write: 12493
Put A to queue...
Process to read: 12494
Get A from queue.
Put B to queue...
Get B from queue.
Put C to queue...
Get C from queue.


In [42]:
from multiprocessing import Pipe, Process

def worker(conn):
    print(conn.recv())
    conn.send("sent from child process")

conn1, conn2 = Pipe()
process = Process(target=worker, args=[conn2])
process.start()

conn1.send("sent from main process")
print(conn1.recv())


sent from main process
sent from child process


In [47]:
import sys
print(sys.maxsize)

9223372036854775807


In [10]:
import math
math.log2(8)

3.0